In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler as StandardScaler

In [2]:
%%capture
'''# import warnings filter (for irgnoring future warning for normalization in sklearn model)
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)'''

In [3]:
# dataset origin: https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge
# load dataset
df= pd.read_csv('/kaggle/input/car-price-prediction-challenge/car_price_prediction.csv')

# Inspect and Clean Dataset

In [4]:
df.shape

In [5]:
df.head()

In [6]:
# remove duplicate rows, if any
df = df.drop_duplicates()
df.shape

In [7]:
df.describe()

In [8]:
df.isnull().any()

In [9]:
df.info()

1) Columns Levy, Engine volume and Mileage would be expected to be numeric -> search for non-numeric values
2) Doors: dates need to be replaced with plausible values

In [10]:
# as could be seen by the .head(), some entries in Levy are '-', which we will interpret as 0
df['Levy'] = df['Levy'].replace('-', 0)
df['Levy'] = pd.to_numeric(df['Levy'], errors='coerce')
#df.info()  # check, if conversion worked

In [11]:
df['Engine volume']

In [12]:
# because of the string ' Turbo' this column is not numeric. Instead of just removing the string, we add a new column to encode
# this information as a categorical variable before removing from column 'Engine volume'
df['Turbo'] = [1 if 'Turbo' in x else 0 for x in df['Engine volume']]
df['Engine volume'] = df['Engine volume'].replace(' Turbo', '', regex=True)
df['Engine volume'] = pd.to_numeric(df['Engine volume'], errors='coerce')
df['Engine volume'].isnull().any()  # check, if conversion to numeric column worked or if there are more strings to take care of

In [13]:
# clean Mileage column, as ' km' was present in the cells displayed with .head()
df['Mileage'] = df['Mileage'].replace(' km', '', regex=True)
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')
df['Mileage'].isnull().any()  # check, if conversion to numeric column worked or if there are more strings to take care of

In [14]:
df['Doors'].value_counts()

In [15]:
# there are 3 values, which are interpreted as dates. 
# 02-Mar is most likely 2-3, 04-May is most likely 4-5, >5 makes sense
df.loc[df['Doors'] == '04-May', 'Doors'] = '4-5'
df.loc[df['Doors'] == '02-Mar', 'Doors'] = '2-3'
df['Doors'].value_counts()

In [16]:
# replace production year with age of car instead by year of publication - prodYear (no publication date found, 2021 is assumed)
df['Age'] = 2021 - df['Prod. year']
df['Age']

In [17]:
df = df.drop(['ID', 'Prod. year'], axis=1)

In [18]:
df.info()

In [19]:
df.describe(())

In [20]:
df.hist()

In [21]:
df['Category'].value_counts()

In [22]:
df = df.drop(df[df['Category'] == 'Goods wagon'].index)  # remove goods wagons as we are only interested in passenger cars

In [23]:
df['Fuel type'].value_counts()

In [24]:
# there is only 1 hydrogen car. To avoid outliers, we remove this one car
df = df.drop(df[df['Fuel type'] == 'Hydrogen'].index)

In [25]:
df[df['Cylinders'] > 10].sort_values(by='Cylinders', ascending=False)

In [26]:
# from the histogram it can be seen that there are cars with 16 cylinders. This is worth having a closer look.
# Thus it was found quite an amount of cars with > 10 cylinders are not plausible. 
# e.g. hatchbacks or Honda or Kia sedans are not plausible. Engine volumes, which are technically closely related to cylinder
# count, however, look plausible.
# we will thus use this as a proxy to calculate more plausible cylinder numbers: 
#(often cylinder volumes are in the order of about 0.5 litres)

df['Vol per cylinder'] = df['Engine volume'] / df['Cylinders']
df[df['Vol per cylinder'] > 0.75]['Price'].count()  # we will take 0.75 as the threshold for the date we clean (0.5 plus 50%)
df[df['Vol per cylinder'] == 0]['Price'].count()  # we will also replace 0 cylinders accordingly as there are only cars with ICE left in the dataset

In [27]:
# the 222 vehicles found above, we will calculate a plausible cylinder number based on the engine volume and round to full cylinders
df_cyl_clean = df[df['Vol per cylinder'] > 0.75].copy()
df_cyl_clean = pd.concat([df_cyl_clean, df[df['Vol per cylinder'] == 0]])
df = df.drop(df[df['Vol per cylinder'] > 0.75].index)
df = df.drop(df[df['Vol per cylinder'] == 0].index)
df_cyl_clean['Cylinders'] = np.ceil(df_cyl_clean['Engine volume'].div(0.5).div(2)).mul(2)  # ceil(num / 2) * 2 to round to closest even number as cylinder numbers are far more often even than odd 
df = pd.concat([df, df_cyl_clean])
df = df.drop(['Vol per cylinder'], axis=1)
df

In [28]:
# as can be seen, std is an order of magnitude higher than the mean. the max looks rather unplausible as well.
# will restrict our dataset to the 99,5% quantile to remove those outliers
#df['Mileage'].hist()
df = df[df['Mileage'] <= df['Mileage'].quantile(0.995)]
df.describe()

In [29]:
#mileage_below_100_df = 
df[df['Mileage'] <100]
#mileage_below_100_not_new_df = mileage_below_100_df[mileage_below_100_df['Age'] >3]

In [31]:
# Many cars with Mileage of 0 even after several years. We will replace the 0 with the product of their age and the
# mean mileage for all cars which are older than 3 years
df['avg_annual_mileage'] = df['Mileage'] / df['Age']
#df.sort_values(by='avg_annual_mileage', ascending=False)
df['avg_annual_mileage']

In [32]:
#corr_mileage_df['Mileage'] = mileage_below_100_not_new_df['Age'] * df['avg_annual_mileage'].mean().round()
mean_mileage_per_year = df['avg_annual_mileage'].mean().round()

In [ ]:
df.loc[(df['Mileage'] < 100) & (df['Age'] > 3), df['Mileage']] = df['Age'] * mean_mileage_per_year
#df = df.drop(mileage_below_100_not_new_df.index)
#df = pd.concat([df, corr_mileage_df])
df.drop(['avg_annual_mileage'], axis=1)
df

# Analysis

In [ ]:
# correlation matrix for numeric colums only
sns.heatmap(df.corr(), annot=True, fmt=".2f");

In [ ]:
# above correlation matrix confirms the commonly known fact that prices for ordinary cars usually decline with the age of a car.
# thus the next few evaluations build on this knowledge

In [ ]:
cars_per_year = df.groupby('Age').count()['Price']
cars_per_year

In [ ]:
df.groupby('Age').mean().astype(int)['Price']

In [ ]:
ax = sns.lineplot(data=cars_per_year, ci=None)
ax2=ax.twinx()
sns.lineplot(x='Age', y='Price', data=df, ax=ax2, color='g', ci=None)
ax.xlabel='Age'
ax.ylabel='Vehicles'
ax2.ylabel='Price'
plt.legend(labels=['Vehicles', 'Mean Price per Vehicle'])

In [ ]:
# we will restrict our analysis to cars <30 years. Cars older than 30 years are hardly relevant for the average buyer and 
# vintage car prices would be worth a separate analysis (which this dataset is not suited for)
# thus we also remove the peaks of of mean prices after 30+ years, caused by outliers of too few cars in the dataset for these ranges.
df = df[df['Age'] <=30]  # remove cars <30 years for the reasons mentioned above
sns.lineplot(x='Age', y='Price', data=df)

In [ ]:
df = df[df['Price'] <= df['Price'].quantile(0.99)]  # eliminate outliers (excessively expensive cars) to focus on the main market
df = df[df['Price'] >= df['Price'].quantile(0.01)]  # eliminate outliers (excessively cheap cars) to focus on the main market

In [ ]:
df.hist()

In [ ]:
col_name = df.columns.to_list()
for col in df.select_dtypes(exclude=['object']).columns.to_list():
    col_name.remove(col)
col_name.remove('Manufacturer')
col_name.remove('Model')
#col_name = ['Fuel type', 'Category']
for col in col_name:
    plt.figure()
    series = df[col].unique().tolist()
    for entry in series:
        sns.lineplot(x='Age', y='Price', data=df[df[col] == entry], ci=None)
    plt.title(label='Mean Car Prices vs. Age by ' + col)
    plt.legend(labels=series)

In [ ]:
# encode numerical variables
#df_before_encoding = df.copy()  # store for subsequent analysis of unencoded df
df_cat = df.select_dtypes(include=['object']).copy()
df_cat = df_cat.drop('Model', axis=1)  # remove column Model to not drastically overfit
df_cat = df_cat.drop('Manufacturer', axis=1)  # remove column Manufacturer to not drastically overfit
df_cat = pd.get_dummies(df_cat, drop_first=True)
df_cat.shape

In [ ]:
# with 40 columns (due to encoding) and 17315 rows, our dataset is big enough to satisfy the rule of thumb of >10 rows per column
df = df.drop(df.select_dtypes(include=['object']).columns, axis=1)
df = pd.concat([df, df_cat], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(40,40))
sns.heatmap(df.corr(), annot=True, fmt=".2f");

In [ ]:
# as can be seen from the matrix, not all features are uncorrelated to one-another, 
# e.g. Jeep and Diesel habe a relatively strong correlation (both of which have a strong correlation with price)

In [ ]:
df.corr()['Price'].sort_values(key=abs, ascending=False)

In [ ]:
#0) create vectors and split dataset
X = df.drop(['Price'], axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

#1) Instantiate
#lm_model = LinearRegression(normalize=True) 
lm_model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

#2) Fit
lm_model.fit(X_train, y_train)

#3) Predict
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)

#4) Score
r2_test = r2_score(y_pred=y_test_preds, y_true=y_test)  # Rsquared
r2_train = r2_score(y_pred=y_train_preds, y_true=y_train)  # Rsquared

In [ ]:
r2_train

In [ ]:
sns.regplot(x=y_train, y=y_train_preds)

In [ ]:
r2_test

In [ ]:
sns.regplot(x=y_test, y=y_test_preds)

In [ ]:
# the model even provides negative prices